In [1]:
!pip install pyprind

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
import pyprind 
import seaborn as sns
from pylab import rcParams
from collections import defaultdict
from scipy.optimize import curve_fit
from collections import Counter
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tqdm_notebook as tqdm

     \ 471kB 4.2MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-6nqszzx3/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)

In [0]:
def read_caps(fname):
    """Load the captions into a dataframe"""
    vn = []
    cap = []
    df = pd.DataFrame();
    with open(fname) as f:
        for line in f:
            pairs = line.split()
            vn.append(pairs[0])
            cap.append(pairs[1])
        df['video']=vn
        df['caption']=cap
    return df

In [0]:
# load the captions
caption_file = '/content/drive/My Drive/CA684_Assignment_Data/dev-set/dev-set_video-captions.txt'
df_cap = read_caps(caption_file)


In [0]:
# load the ground truth values
ground_truth_file = '/content/drive/My Drive/CA684_Assignment_Data/dev-set/dev-set_ground-truth.csv'
ground_truth = pd.read_csv(ground_truth_file)

print(ground_truth)


In [9]:
from string import punctuation
counts = Counter()
# setup prograss tracker
pbar = pyprind.ProgBar(len(df_cap['caption']), title='Counting word occurrences')
for i, cap in enumerate(df_cap['caption']):
    # replace punctuations with space
    # convert words to lower case 
    text = ''.join([c if c not in punctuation else ' ' for c in cap]).lower()
    df_cap.loc[i,'caption'] = text
    pbar.update()
    counts.update(text.split())

Counting word occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [10]:
from keras.preprocessing.text import Tokenizer
# build the word index
len_token = len(counts)
tokenizer = Tokenizer(num_words=len_token)
print(len_token)


5191


Using TensorFlow backend.


In [11]:
tokenizer.fit_on_texts(list(df_cap.caption.values)) #fit a list of captions to the tokenizer
#the tokenizer vectorizes a text corpus, by turning each text into either a sequence of integers

print(len(tokenizer.word_index))


5191


In [0]:
one_hot_res = tokenizer.texts_to_matrix(list(df_cap.caption.values),mode='binary')
sequences = tokenizer.texts_to_sequences(list(df_cap.caption.values))

In [13]:
#Just to visualise some stuff in sequences and counts
print(sequences[0]) # prints location of words from caption 0 'blonde woman is massaged tilt down'
print(counts['blonde']) # no. of occurences of 'blonde'
n=3
print('Least Common: ', counts.most_common()[:-n-1:-1])       # n least common elements
print('Most Common: ',counts.most_common(n))

[724, 7, 35, 884, 384, 91]
10
Least Common:  [('unknown', 1), ('handheldshowing', 1), ('longof', 1)]
Most Common:  [('in', 2105), ('the', 1835), ('on', 1724)]


In [0]:
# calculating max length
max_len = 50

In [15]:
print(sequences[0]) # length of 1st sequence

[724, 7, 35, 884, 384, 91]


In [0]:
X_seq = np.zeros((len(sequences),max_len))
for i in range(len(sequences)):
    n = len(sequences[i])
    if n==0:
        print(i)
    else:
        X_seq[i,-n:] = sequences[i]
X_seq.shape

X_seq[0,:]

Y = ground_truth[['short-term_memorability','long-term_memorability']].values
X = X_seq # sequences

In [165]:
X = one_hot_res;
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

X_train.shape

(4800, 5191)

In [166]:
!pip list | grep -i keras


Keras                    2.2.4                
Keras-Applications       1.0.7                
Keras-Preprocessing      1.0.9                
keras-vis                0.4.1                


In [167]:
pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [168]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=scaler.fit_transform(X)
X

array([[ 0.        , -0.66193592, -0.55989741, ..., -0.01291102,
        -0.01291102, -0.01291102],
       [ 0.        ,  1.51072026, -0.55989741, ..., -0.01291102,
        -0.01291102, -0.01291102],
       [ 0.        , -0.66193592, -0.55989741, ..., -0.01291102,
        -0.01291102, -0.01291102],
       ...,
       [ 0.        ,  1.51072026, -0.55989741, ..., -0.01291102,
        -0.01291102, -0.01291102],
       [ 0.        , -0.66193592, -0.55989741, ..., -0.01291102,
        -0.01291102, -0.01291102],
       [ 0.        ,  1.51072026, -0.55989741, ..., -0.01291102,
        -0.01291102, -0.01291102]])

In [65]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=0)
# pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=0, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt


In [170]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

Y_train.reshape



<function ndarray.reshape>

In [0]:
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score

knn = KNeighborsRegressor(n_neighbors=10, p=2, metric="euclidean")
history = knn.fit(X_train,Y_train)

Y_pred = knn.predict(X_test)

In [172]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

Get_score(Y_pred, Y_test)

The Spearman's correlation coefficient is: 0.338
The Spearman's correlation coefficient is: 0.090
